# Hands-On Text Models and Logistic Regression 
***

In this notebook we'll explore Scikit-Learn's ability to create features from text, including how to use the Bag-of-Words model.  We'll also explore a more sophisticated feature creation technique called TFIDF.  Finally, we'll use Scikit-Learn's Logistic Regression classifier to make semantic predictions about movie reviews. 

In [ ]:
import numpy as np
import matplotlib.pylab as plt 
%matplotlib inline 

### Part 1: The Bag-of-Words Text Model 
***

In the first portion of this notebook we'll explore the bag of words text model.  In lecture we worked with the following corpus of documents related to newspaper titles: 

$\texttt{Training Set}:$

$\texttt{d1}: \texttt{new york new tribune}$

$\texttt{d2}: \texttt{new york times}$

$\texttt{d3}: \texttt{los angeles times}$

First we'll define the vocabulary based on the words in the test set.  It is $V = \{ \texttt{angeles}, \texttt{los}, \texttt{new}, \texttt{times}, \texttt{tribune}, \texttt{york}\}$. 

We need to define an association between the particular words in the vocabulary and the specific entries in our vectors.  Let's define this association in the order that we've listed them above.  We can store this mapping as a Python dictionary as follows: 

In [ ]:
V = {'angeles': 0, 'los': 1, 'new': 2, 'times': 3, 'tribune': 4, 'york': 5}

Let's also store the documents in a list as follows: 

In [ ]:
D = ["new york new tribune", "new york times", "los angeles times"]

To be consistent with sklearn conventions, we'll encode the documents as *row-vectors* stored in a **Document-Term Matrix**.  In this case, each row of the matrix corresponds to a document, and each column corresponds to a term in the vocabulary.  For our example this gives us a matrix `Xdt` of shape $3 \times 6$.  The $(d,t)$-entry in `Xdt` is then the number of times the term $t$ appears in document $d$

**Part A**: Your first task is to write some simple Python code to construct the document-term matrix `Xdt`

In [ ]:
Xdt = np.zeros((len(D),len(V)))

for ii, doc in enumerate(D): 
    for term in doc.split(): 
        Xdt[0,0] += 1 # TODO 
        
print(Xdt)

Hopefully your code returns the matrix 

$$\texttt{Xdt} = 
\left[
\begin{array}{ccccccc}
0 & 0 & 2 & 0 & 1 & 1 \\
0 & 0 & 1 & 1 & 0 & 1 \\
1 & 1 & 0 & 1 & 0 & 0 \\
\end{array}
\right]$$.  

Note that the entry in the (2,0) position is $1$ because the first word (angeles) appears once in the third document. Similarly the entry in the (0,2) position is $2$ because the third word (new) appears twice in the first document. 

OK, let's see how we can construct the same document-term matrix in sklearn.  We will use something called the <a href="http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html">CountVectorizer</a> to accomplish this. Let's see some code and then we'll explain how it functions. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer    # import CountVectorizer 
bagofwords = CountVectorizer()                                 # initialize the vectorizer
Xbw = bagofwords.fit_transform(D)                              # fit to training data and transform to matrix 

The $\texttt{fit_transform}$ method actually does two things.  It fits the model to the training data by building a vocabulary.  It then transforms the text in $D$ into matrix form.  

If we wish to see the vocabulary you can do it like so 

In [ ]:
print(bagofwords.vocabulary_)

Note that this is the same vocabulary and indexing that we defined ourselves.  Hopefully that means we'll get the same term-frequency matrix.  We can print `Xbw` and check 

In [ ]:
print(Xbw.todense())

Yep, they're the same!  Notice that we had to convert $X$ to a dense matrix for printing.  This is because CountVectorizer actually returns a sparse matrix.  This is a very good thing since most vectors in a text model will be **extremely** sparse, since most documents will only contain a handful of words from the vocabulary. 


OK, now suppose that we have a query document not included in the training set that we want to vectorize.  

In [ ]:
d4 = ["new york post"]

We've already fit the CountVectorizer to the training set, so all we need to do is transform the test set documents into a term-frequency vector using the same conventions.  Since we've already fit the model, we do the transformation with the $\texttt{transform}$ method: 

In [ ]:
x4 = bagofwords.transform(d4)

Let's print it and see what it looks like 

In [ ]:
print(x4.todense())

**Part B**: What's missing from `x4` that we might expect to see from the query document? 

<br>

### Part 2: Classifying Semantics in Movie Reviews
***
> The data for this problem was taken from the <a href="https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words">Bag of Words Meets Bag of Popcorn</a> Kaggle competition

In this problem you will use the text from movie reviews to predict whether the reviewer felt positively or negatively about the movie using a Bag-of-Words text model. I've partially cleaned the data and stored training and validation sets in `data/movieReviews/movieTrainData.tsv` and `data/movieReviews/movieValidationData.tsv`.  We'll load the data and process it a bit more in the following cell. 

In [ ]:
import pandas as pd

def read_and_clean_data(fname, remove_stops=True):
    
    # Read in text and labels from file 
    df = pd.read_csv(fname, sep='\t', header=None)
    df.columns = ["labels", "text"]
    
    # Make everything lowercase
    df.loc[:,"text"] = df["text"].apply(lambda s: s.lower())
    
    return df["text"].values, df["labels"].values

In [ ]:
text_train, labels_train = read_and_clean_data("data/movieReviews/movieTrainData.tsv")
text_valid, labels_valid = read_and_clean_data("data/movieReviews/movieValidationData.tsv")

**Part A**: Look at a few of the reviews stored in $\texttt{text_train}$ as well as their associated labels in $\texttt{labels_train}$.  Can you figure out which label refers to a positive review and which refers to a negative review? 

OK, the first thing we'll do is train a logistic regression classifier using the Bag-of-Words model, and see what kind of accuracy we can get.  To get started, we need to vectorize the text into numerical features that we can use.  We'll use CountVectorizer to turn our text into features, Scikit-Learn's [LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier to do classification, and a pipeline to tie everything together. First though, we'll fit our CountVectorizer independently so we can learn more about the data. 

In [ ]:
bagofwords = CountVectorizer()
Xbw_train = bagofwords.fit_transform(text_train)

**Part B**: How many different terms are in the vocabulary? 

**Part C**: What are the most common words in the corpus? 

In [ ]:
def most_common_words(X, vocab, num_common=20):
    # TODO
    for ii in range(num_common):
        word_freq, word = word_freqs[ii]
        print("{:12s} {:d}".format(word, word_freq))

most_common_words(Xbw_train, bagofwords.vocabulary_)

It looks like we have some overly common stop words that are probably not useful for classification.  We can remove them by passing in the `stop_words` parameter to the `CountVectorizer`.  We need to tell it which language we're using. How many words are left after removing the stop words? 

**Part D**: Next we'll build a pipeline that turns text into features using CountVectorizer and then performs classification using LogisticRegression. 

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.linear_model import LogisticRegression
bagofwords_steps = [
    ("bagofwords", CountVectorizer(stop_words="english")),
    ("logreg", LogisticRegression())
]
bagofwords_pipe = Pipeline(bagofwords_steps)

We can fit our model to the training data using the `.fit()` method. 

In [ ]:
bagofwords_pipe.fit(text_train, labels_train)

Now let's make some predictions.  The following code will print a message form the validation set, along with it's true label, and then make a prediction using the pipeline. Test it out for a few validation examples by changing the value of `ind`.  How does the classifier seem to be doing? 

In [ ]:
ind = 1 
print("text labeled {:d}:\n".format(labels_valid[ind]))
print(text_valid[ind])
pred = bagofwords_pipe.predict([text_valid[ind]])
print("\npredicted label: {:d}".format(pred[0]))

**Part D**: Finally, let's make predictions on the the entirety of both the training and the validation sets and compute overall accuracies.  We could do this by hand, but LogisticRegression (like most models in Scikit-Learn) includes a `.score()` method which takes in data and the true labels, makes predictions, and then computes the accuracy.  Since LogisticRegression is part of our pipeline, our pipeline inherits this score function. 

In [ ]:
train_acc = bagofwords_pipe.score(text_train, labels_train)
valid_acc = bagofwords_pipe.score(text_valid, labels_valid)
print("Accuracy on Training Set:   {:.3f}".format(train_acc))
print("Accuracy on Validation Set: {:.3f}".format(valid_acc))

It looks like we're getting $100\%$ accuracy on the training set (which should worry you) and around a $79\%$ on the validation set.  Clearly, our features are overfitting the training data.  Let's see if regularization would be helpful.  The `LogisticRegression` method takes a parameter `C` for regularization strength.  The default setting is `C=1`. Try increasing this value and see if the validation accuracy improves. 

### Part 3: Term Frequency - Inverse Document Frequency 
***

The Bag-of-Words model for text classification is very popular, but let's see if we can do better.  Currently we're weighting every word in the corpus by it's frequency.  It turns out that in text classification there are often features that are not particularly useful predictors for the document class, either because they are too common or too uncommon.  **Stop-words** are extremely common, low-information words like "a", "the", "as", etc.  Removing these from documents is typically the first thing done in peparing data for document classification. 



Other words that tend to be uninformative predictors are words that appear very very rarely.  In particular, if they do not appear frequently enough in the training data then it is difficult for a classification algorithm to weight them heavily in the classification process. 

In general, the words that tend to be useful predictors are the words that appear frequently, but not too frequently. 
Another common model for identifying the useful terms in a document is the Term Frequency - Inverse Document Frequency (tf-idf) model.  Here we won't throw away any terms, but we'll replace their Bag-of-Words frequency counts with tf-idf scores which we describe below. 

The tf-idf score is the product of two statistics, *term frequency* and *inverse document frequency*


$$\texttt{tfidf(d,t)} = \texttt{tf(d,t)} \times \texttt{idf(t)}$$

The term frequency $\texttt{tf(d,t)}$ is a measure of the frequency with which term $t$ appears in document $d$.  The inverse document frequency $\texttt{idf(t)}$ is a measure of how much information the word provides, that is, whether the term is common or rare across all documents.  By multiplying the two quantities together, we obtain a representation of term $t$ in document $d$ that weighs how common the term is in the document with how common the word is in the entire corpus. You can imagine that the words that get the highest associated values are terms that appear many times in a small number of documents. 


There are many ways to compute the composite terms $\texttt{tf}$ and $\texttt{idf}$.  For simplicity, we'll define $\texttt{tf(d,t)}$ to be the number of times term $t$ appears in document $d$ (i.e., Bag-of-Words). We will define the inverse document frequency as follows: 

$$
\texttt{idf(t)} = \log \left(~ \frac{1 + \textrm{total # documents}}{1 + \textrm{# documents with term }t}\right) + 1
 = \log \left( \frac{1+n_d}{1+\texttt{df}(t)} \right) + 1
$$


**Part A**: Compute $\texttt{idf(t)}$ for the words `tribune` and `new` in our toy training set. 

$\texttt{Training Set}:$

$\texttt{d1}: \texttt{new york new tribune}$

$\texttt{d2}: \texttt{new york times}$

$\texttt{d3}: \texttt{los angeles times}$


**Part B**: Compute the `tfidf` feature values for the word `new` in the first two documents, as well as the word `tribune` in the first document. 

**Part C**: OK, let's construct TFIDF features for our movie reviews, and see if we get a better LogisticRegression classifier.  All we need to do is replace `CountVectorizer` with [`TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) in our pipeline. If you go to the documentation, you'll notice that `TfidfVectorizer` includes many of the same parameters as `CountVectorizer`, including the ability to automatically remove stop words.  

In [ ]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer   

# Build the pipeline 


# Fit model 

# Get accuracies on training and validation set 
print("Accuracy on Training Set:   {:.3f}".format(train_acc))
print("Accuracy on Validation Set: {:.3f}".format(valid_acc))

**Part D**: How'd you do?  Did the TFIDF features perform better on the validation set than regular old Bag-of-Words?  What happened with the training set? 

<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.MathJax nobr>span.math>span{border-left-width:0 !important};
</style>
""")